### Import Packages

In [1]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import time

### Chrome options

In [2]:
options = Options()
options.add_experimental_option("prefs", {
  "download.prompt_for_download": False,
  "safebrowsing.enabled": True,
  "profile.default_content_setting_values.automatic_downloads": 1
})

### Download CSV files

In [3]:
airport_filter = [6,12,28,31,2,21,29,18,3,19,]
#Sun Country Airlines is missing

In [8]:
#Create browser instance and logon to transtats website
browser = webdriver.Chrome(executable_path=r'C:\Users\Selva\Downloads\chromedriver_win32\chromedriver.exe',
                          chrome_options=options)
browser.get('https://www.transtats.bts.gov/ONTIME/Departures.aspx')

#Check all statistics
browser.execute_script('document.getElementById("chkAllStatistics").checked = true')
for i in range(0,8):
    browser.execute_script('document.getElementById("chkStatistics_'+str(i)+'").checked = true')

#Check all days
browser.execute_script('document.getElementById("chkAllDays").checked = true')
for i in range(0,31):
    browser.execute_script('document.getElementById("chkDays_'+str(i)+'").checked = true')


#Get airport and airline counts
airport_count = browser.execute_script('return document.getElementById("cboAirport").options.length')
airline_count = browser.execute_script('return document.getElementById("cboAirline").options.length')

#Iterate over all airports and airlines for a given month and year and download csv data
for i in range(15,airport_count):
    browser.execute_script('document.getElementById("cboAirport").selectedIndex = '+str(i))
    for j in airport_filter:
        browser.execute_script('document.getElementById("cboAirline").selectedIndex = '+str(j))
        for k in range(0,12):
            browser.execute_script('document.getElementById("chkMonths_'+str(k)+'").checked = true')
            for l in range(13,33):
                browser.execute_script('document.getElementById("chkYears_'+str(l)+'").checked = true')
                #click submit
                browser.execute_script('document.getElementById("btnSubmit").click()')
                #wait until page is loaded
                timeout = 2
                element_present = EC.presence_of_element_located((By.ID, 'cboAirport'))
                WebDriverWait(browser, timeout).until(element_present)
                try:
                    #download csv
                    browser.execute_script('document.getElementById("DL_CSV").click()')
                    dl_wait = True
                    seconds = 0
                    #wait until download is complete
                    while dl_wait and seconds < 5:
                        time.sleep(1)
                        dl_wait = False
                        for fname in os.listdir(r'C:\Users\Selva\Downloads'):
                            if fname.endswith('.crdownload'):
                                dl_wait = True
                        seconds += 1
                except:
                    browser.execute_script('document.getElementById("chkYears_'+str(l)+'").checked = false')
                    continue
                browser.execute_script('document.getElementById("chkYears_'+str(l)+'").checked = false')
            browser.execute_script('document.getElementById("chkMonths_'+str(k)+'").checked = false')
browser.close()

C:\Users\Selva\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


JavascriptException: Message: javascript error: Cannot set property 'checked' of null
  (Session info: chrome=77.0.3865.90)
